In [1]:
from gensim import corpora,models,similarities,utils
import jieba
import jieba.posseg as pseg
import re
import datetime
import random

In [2]:
ad_data_list = [['為何現年27歲的美國人特別多？ 讓數據來告訴你','新聞網站「石英」引用美國人口普查局的資料報導，全美現在有460多萬人出生在1991年，人數之多遠超過其他年份，這群現年27歲的人未來很可能像嬰兒潮世代那樣，主宰美國政治、文化及經濟。石英指出，有兩個原因可解釋為何1991年出生的美國人那麼多。首先，在1990年之前，另一個出生人數高峰的年份是1963年，那年出生的人在1991年達到育齡高峰。其次，這群27歲美國人中，約50萬人是移民，這是美國1970年代以來的趨勢之一，即出生在外國的美國人占總人口比例越來越高，不過此趨勢從2010年以來放緩。石英說，現年27歲的這群美國人很快就會展現更多影響力，他們未來可能像出生在1946-1964的嬰兒潮世代那樣，主宰美國政治、文化及經濟。',10],
               ['韓導網軍假新聞舖天蓋地 高雄麥可阿伯拍片反擊','〔記者蔡清華／高雄報導〕熱愛麥可傑克遜的高雄阿伯林敬堯，經常興致一來，就扮起麥可，來一段招牌倒退舞步，自從韓國瑜南漂到高雄參選市長，大量網軍造假當道，無孔不入，連政見會眾目睽睽之下，都可造假別人帶耳機，晚會主持人感謝「大家沒離開」被硬說成哭求「大家嘜離開」！麥可阿伯忍無可忍，跳出來自拍影片重砲回擊，呼籲全民一起來打假！林敬堯說，實在被氣極了，在高雄從年輕到現在3、40年來，看過多少次選舉，經歷各種輸贏，沒有像這次被霸凌得這麼鬱悶又氣憤！一定要讓自己一吐這股怨氣！他一口氣拍了「麥可翻魚」、「騙騙騙篇」、「造假大王篇」「根留台灣篇」等等。在造假大王篇裡，他扮演「被阿港伯附身的麥可」「有人問阿港伯，，這次韓導選舉有什麼特色？我本人認為就只有3個字：假！假！假！」「假影片、假身份、假新聞、假民調、假消息、假帳號 一切攏係假啦！明明是貴族學校的老闆，還假裝窮菜販…」',15],
               ['一張海報洩底！學者：韓國瑜前功盡棄','國民黨高雄市長候選人韓國瑜三山大會戰最後一場14日在岡山登場，預料會有各方支持者到場，據相關報導，屆時國民黨台中市長候選人盧秀燕丶國民黨新北市長候選人侯友宜將參加「岡山大會戰」，韓、盧、侯將合體，另，網路上近日流傳三人岡山大會戰海報，對此，有學者說，這張藍營內部廣傳的海報，同溫層大爆發；只是，韓國瑜原本刻意塑造白色與素人色彩的苦心，前功盡棄，而那個黃復興黨部的軍系立委又回來了。台師大政治所教授范世平在臉書貼出海報表示，非常佩服海報設計者的電腦合成技術，但是把二戰時的螺旋槳飛機與現代化戰車混在一起，是缺乏專業知識還是搞穿越劇？三個大兵頭戴國軍軍帽（事實上國軍沒戴貝雷帽）卻身著美軍軍服，這種混搭頗好笑。他指出，這張殺氣騰騰的海報的確讓藍營熱血沸騰，準備大開殺戒，看得出藍營累積已久的憤怒；但是當韓國瑜與支持者高唱軍歌「夜襲」，加上這張海報，韓國瑜原本刻意塑造白色與素人色彩的苦心，前功盡棄！「而那個黃復興黨部的軍系立委又回來了」。范世平還說，這張海報從造型到用辭非常有「八百壯士」的fu，但中間選民是否會嚇跑？',8],
               ['台灣主要報紙重要財經新聞 2018年11月13日','上市櫃公司 10 月營收在蘋果供應鏈、NB 等族群帶動下，以 3.32 兆元寫下歷史單月新高；累計前十月營收逾 28 兆元，約增一成，同樣改寫歷史新猷。中國國務院總理李克強昨（12）日飛抵新加坡進行國是訪問，將出席第 33 屆東協系列會議及東亞領導人會議。李克強此行最重要的目的，在於加速「區域全面經濟夥伴關係協定（RCEP）」談判於年底前完成。昨日，中星兩方也簽署了「自由貿易協定升級議定書」，為兩國經貿合作全面升級。LINE Bank 股東名單出爐，除 LINE Financial（台灣連線金融科技公司）持股 49.9％最多外，四大金融業股東分別為台北富邦、中國信託、聯邦及渣打銀行，加上台灣大哥大、遠傳兩家電信業者，將聯手爭取「三搶二」的純網銀執照。',15],
               ['小偷跑得比警察快… 立委促查假新聞','選舉越來越靠近，各種選戰謠言也滿天飛，多名交通委員會立委共同對假新聞流竄感到憂心。立委劉世芳昨在交通委員會直指，許多業者透過網路散播不實消息，為特定候選人帶風向，更比喻現在是「小偷跑得快，警察比較慢」，暗喻ＮＣＣ處理假新聞不力。ＮＣＣ主委詹婷怡面對多位立委質詢時強調，有些消息究竟是真還是假可能很難判定，但ＮＣＣ會要求業者落實衛星廣播電視法第廿七條第三項第四款規定，若媒體報導內容與事實不符，損害公共利益，以及未善盡事實查證責任，ＮＣＣ將依法處理。劉世芳痛批，「小偷跑得比較快，警察比較慢」，她先前跟內政部、法務部溝通，官員都說假新聞是ＮＣＣ的責任，行政院秘書長卓榮泰也說這不是單一部會可以處理，政府當然希望媒體自律，但如果有業者收受高額金額釋放假訊息，效果就像在戲院裡面大喊失火一樣。劉世芳表示，她了解假新聞非單一部會的業務，但希望ＮＣＣ不要再推來推去，現在假訊息滿天飛，甚至有中國網軍謠傳總統蔡英文要把太平島賣給美國，這樣的網路謠言完全沒人監管，更問詹婷怡是否要比照美國、日本和法國訂定相關法令管理假訊息來源。立委鄭寶清也表示，衛廣法第廿七條也明確要求媒體刊登前必須查證',5]]

In [3]:
data_context = '隨著蘋果宣佈不再公開產品銷量，加上停止擴產的消息一再傳出，外界近期一致憂心蘋果 iPhone 銷量，甚至包括原先預計會熱賣的 iPhone XR。指標性的知名分析師郭明錤，便罕見下修 iPhone XR 在今年最後一季的下量，幅度達 30% ~ 35%，約是 3,000 ~ 3,500 萬支；明年上半年 iPhone XR 的出貨預期亦有 25% ~ 30% 的下修。郭明錤認為，iPhone XR 的銷量下修，與美中貿易戰、定價偏高和華為競爭有關。Face ID 的關鍵 3D 感測器供應商 Lumentum 也下修財報預期近 25%，原因是「大客戶大幅減少先前訂單的出貨量」。雖然沒提到客戶是誰，但蘋果實際上正是 Lumentum 最大的下單者。為 iPhone XR 供應 LCD 面板的日本顯示器（JDI），也下調未來兩季度的營收預期，可能暗示 iPhone XR 的銷量沒有預期中突出。摩根大通稍早出具的報告，也下調了蘋果營收的預期，同時推估 iPhone 的銷量將會比去年同期下滑。花旗也發現人們對新 iPhone 的關注正在下滑。市場狀況亦顯示，蘋果可能面臨了 2015 年發表 iPhone 6S 後，再一次的銷量下跌情形。儘管隨後發表了較平價的 iPhone SE，蘋果仍在該財年首次面臨 iPhone 銷量下跌。調研機構「Longbow Research」則稱，蘋果的供應鏈重心，也正在由新機轉往舊機。該報告並稱，iPhone XR 和 XS Max 的訂單分別被削減了 20% ~ 30%，但 iPhone 8 與 8 Plus 等舊機，則反向增加 20% ~ 25%。郭明錤也談到因為降價，蘋果舊機的需求大幅增加，加上 iPhone XS 系列銷量較預期更好，有望彌補 XR 流失的客群。因此，郭明錤最終仍維持今年最後一季整體 iPhone 出貨預期為 7,500 萬 ~ 8,000 萬支，沒有改動，但明年首季的銷量則仍未被看好，下修 15% ~ 20% 至 4,700 萬 ~ 5,200 萬台，較去年同期約少了 500 萬支。'

In [4]:
ad_context_list = []
for i in range(len(ad_data_list)):
    ad_context_list.append(ad_data_list[i][1])

In [5]:
jieba.load_userdict("/Users/jack/Desktop/LDA/dict.txt.big")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/z0/0vtl52ys4js7_0rnmqt9bdfr0000gt/T/jieba.cache
Loading model cost 1.177 seconds.
Prefix dict has been built succesfully.


In [6]:
#移除括號內的內容
remover1=re.compile(r"\（.*?\）")
remover2=re.compile(r"\[.*?\]")
remover3=re.compile(r"\〔.*?\〕")
remover4=re.compile(r"\(.*?\)")

data_context=remover1.sub("",data_context)
data_context=remover2.sub("",data_context)
data_context=remover3.sub("",data_context)
data_context=remover4.sub("",data_context)

In [7]:
for i in range(len(ad_context_list)):
        ad_context_list[i]=remover1.sub("",ad_context_list[i])
        ad_context_list[i]=remover2.sub("",ad_context_list[i])
        ad_context_list[i]=remover3.sub("",ad_context_list[i])
        ad_context_list[i]=remover4.sub("",ad_context_list[i])

In [8]:
with open('/Users/jack/Desktop/LDA/stopword.txt', encoding='utf-8') as f:
    stop_word_list = f.readlines()
    
stop_word_list = [x.strip() for x in stop_word_list]

In [9]:
lda_jieba_doc = []
for i in data_context:
    tokens = jieba.cut(i,cut_all=False,HMM=True)
    stopped_tokens = [i for i in tokens if not i in stop_word_list]
    lda_jieba_doc.append(stopped_tokens)

In [10]:
text_list = []
for i in ad_context_list:
    tokens = jieba.cut(i,cut_all=False,HMM=True)
    stopped_tokens = [i for i in tokens if not i in stop_word_list]
    text_list.append(stopped_tokens)

In [11]:
# load dictionary
dictionary = corpora.Dictionary.load('/Users/jack/Desktop/LDA/allnews_dictionary.dict')

In [12]:
# 建立語料庫
corpus = [dictionary.doc2bow(text) for text in text_list]

In [13]:
tfidfModel = models.tfidfmodel.TfidfModel.load("/Users/jack/Desktop/LDA/tfidf_model.tfidf")

In [14]:
# 轉為向量表示
tfidfVectors = tfidfModel[corpus]

# load lda model
ldamodel = models.ldamodel.LdaModel.load('/Users/jack/Desktop/LDA/allnews_LDA_model.lda')

corpus_lda = ldamodel.get_document_topics(tfidfVectors ,minimum_probability=0)
# 建立索引
#indexLDA = similarities.MatrixSimilarity(ldamodel[tfidfVectors])
indexLDA = similarities.MatrixSimilarity(corpus_lda)

/Users/jack/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [15]:
# 以新聞內容作為比較基準

ad_sims_list = indexLDA[ldamodel[dictionary.doc2bow(data_context.split())]]
# 排列相似度由大至小
ad_sims_list = sorted(enumerate(ad_sims_list), key=lambda item: -item[1])
print(ad_sims_list)

[(3, 0.87258047), (1, 0.8428356), (0, 0.71053857), (4, 0.6598892), (2, 0.60823804)]


In [16]:
hit_time = datetime.datetime.now()
ad_list = []
for i in range(len(ad_sims_list)):
	ad_index = ad_sims_list[i][0]
	ad_title = ad_data_list[ad_index][0]
	ad_sims = ad_sims_list[i][1]
	ad_amount = ad_data_list[ad_index][2]
	lda_ad = corpus_lda[ad_index]
	ad_list.append([ad_index, ad_title, ad_sims, ad_amount, lda_ad, hit_time])

In [17]:
user_list = [['ajacka','_'],['kjacka','_'],['tjacka','_'],['eejacka','_'],['oojacka','_'],['yyjacka','_'],['1jacka','_'],['jac1ka','_'],['jacka20','_'],['jacka1','_']
            ,['bjacka','_'],['ljacka','_'],['ujacka','_'],['ffjacka','_'],['ppjacka','_'],['zzjacka','_'],['2jacka','_'],['jac2ka','_'],['jacka19','_'],['jacka2','_']
            ,['cjacka','_'],['vjacka','_'],['wjacka','_'],['ggjacka','_'],['qqjacka','_'],['1ajacka','_'],['3jacka','_'],['jac3ka','_'],['jacka18','_'],['jacka3','_']
            ,['djacka','_'],['mjacka','_'],['xjacka','_'],['hhjacka','_'],['rrjacka','_'],['1bjacka','_'],['4jacka','_'],['jac4ka','_'],['jacka17','_'],['jacka4','_']
            ,['ejacka','_'],['njacka','_'],['yjacka','_'],['iijacka','_'],['ssjacka','_'],['1cjacka','_'],['5jacka','_'],['jac5ka','_'],['jacka16','_'],['jacka5','_']
            ,['fjacka','_'],['ojacka','_'],['zjacka','_'],['jjjacka','_'],['ttjacka','_'],['1djacka','_'],['6jacka','_'],['jac6ka','_'],['jacka15','_'],['jacka6','_']
            ,['gjacka','_'],['pjacka','_'],['aajacka','_'],['kkjacka','_'],['uujacka','_'],['1ejacka','_'],['7jacka','_'],['jac7ka','_'],['jacka14','_'],['jacka7','_']
            ,['hjacka','_'],['qjacka','_'],['bbjacka','_'],['lljacka','_'],['vvjacka','_'],['1fjacka','_'],['8jacka','_'],['jac8ka','_'],['jacka13','_'],['jacka8','_']
            ,['ijacka','_'],['rjacka','_'],['ccjacka','_'],['mmjacka','_'],['wwjacka','_'],['1gjacka','_'],['9jacka','_'],['jac9ka','_'],['jacka12','_'],['jacka9','_']
            ,['jjacka','_'],['sjacka','_'],['ddjacka','_'],['nnjacka','_'],['xxjacka','_'],['1hjacka','_'],['10jacka','_'],['jac10ka','_'],['jacka11','_'],['jacka10','_']]

In [18]:
#設定門檻值,超過門檻值進行第一次的配對
threshold = 0.75

select_ad_list = []
not_select_ad_list = []


for i in range(len(ad_list)):
	if ad_list[i][2] >= threshold:
		select_ad_list.append(ad_list[i])
	else:
		not_select_ad_list.append(ad_list[i])

In [19]:
#排序相關性由高至低
select_ad_list = sorted(select_ad_list, key=lambda x:x[2], reverse=True)

In [20]:
user_list_token = []
for i in range(len(user_list)):
    user_list_token.append(user_list[i][0])

In [21]:
for select_ad in select_ad_list:

    if select_ad[3] == 0:
        break
    else:
        if len(user_list_token) > select_ad[3]:
            select_user_for_ad = random.sample(user_list_token,select_ad[3])
            select_ad.append(select_user_for_ad)
            user_list_token = list(set(user_list_token) - set(select_user_for_ad))
            select_ad[3] = int(select_ad[3]) - len(select_user_for_ad)
        else:
            select_ad.append(user_list_token)
            select_ad[3] = int(select_ad[3]) - len(user_list_token)
            user_list_token = []

In [22]:
#媒合剩餘廣告
if user_list != []:
    #增加在list內的index
    not_select_ad_list_with_index = list(enumerate(not_select_ad_list))
    for user in user_list:
        user_model = user[1]
        if user_model != '_':

            for re_select_ad in not_select_ad_list_with_index:
                re_select_ad_result = []
                #判斷廣告是否還有額度
                if re_select_ad[1][3] == 0:
                    break
                else:
                    #轉換為浮點數再放入model        
                    user_model = [float(coef_i) for coef_i in user[1].split(',')]
                    lda_select_ad = [float(ad_i) for ad_i in re_select_ad[1][4].split(',')]
                    [category_hour, category_week] = LR_function.categorize_time(re_select_ad[1][5])
                    hit_time = [category_hour[:-1] + category_week[:-1]]
                    prob = LR_function.prediction(user_model, lda_select_ad, hit_time)
                    re_select_ad_result.append([re_select_ad[0], prob])
                    #排序廣告與User之相關性
                    re_select_ad_result = sorted(re_select_ad_result, key=lambda x:x[1], reverse=True)
                    #最佳ad的index
                    best_select_ad = re_select_ad_result[0]
                    if len(not_select_ad_list_with_index[best_select_ad][1]) > 6:
                        not_select_ad_list_with_index[best_select_ad][1][6].append(user)
                    else:
                        not_select_ad_list_with_index[best_select_ad][1].append([user])

                    not_select_ad_list_with_index[best_select_ad][1][3] = int(not_select_ad_list_with_index[best_select_ad][1][3]) - 1
                    user_list.remove(user)            

        #若沒有模型則隨機挑選廣告給User
        else:
            random_select_ad = random.choice(not_select_ad_list_with_index)
            #判斷廣告是否還有額度
            if random_select_ad[1][3] == 0:
                #重新隨機選擇廣告
                random_select_ad = random.choice(not_select_ad_list_with_index)
            else:
                if len(not_select_ad_list_with_index[random_select_ad[0]][1]) > 6:
                    not_select_ad_list_with_index[random_select_ad[0]][1][6].append(user)
                else:
                    not_select_ad_list_with_index[random_select_ad[0]][1].append([user])


                random_select_ad[1][3] = int(random_select_ad[1][3]) - 1
                user_list.remove(user)
else:
    pass

In [23]:
#轉換為原來格式
need_select_ad_list = []
for i in range(len(not_select_ad_list_with_index)):
    need_select_ad_list.append(not_select_ad_list_with_index[i][1]) 

In [24]:
#將兩次媒合的結果結合
push_ad_list = []
all_ad_list = select_ad_list + need_select_ad_list
for ad in all_ad_list:
    if len(ad) > 6:
        push_ad_list.append(ad)

In [25]:
push_ad_list

[[3,
  '台灣主要報紙重要財經新聞 2018年11月13日',
  0.87258047,
  0,
  [(0, 0.030372985),
   (1, 0.034716986),
   (2, 0.0014455208),
   (3, 0.0014455208),
   (4, 0.0014455208),
   (5, 0.0014455208),
   (6, 0.0014455208),
   (7, 0.020581882),
   (8, 0.0014455208),
   (9, 0.0014455208),
   (10, 0.0014455208),
   (11, 0.0014455208),
   (12, 0.015543107),
   (13, 0.010055002),
   (14, 0.0014455208),
   (15, 0.0456843),
   (16, 0.0014455208),
   (17, 0.027984055),
   (18, 0.0014455208),
   (19, 0.0014455208),
   (20, 0.0014455208),
   (21, 0.0014455208),
   (22, 0.0014455208),
   (23, 0.0014455208),
   (24, 0.080192894),
   (25, 0.0014455208),
   (26, 0.0014455208),
   (27, 0.0014455208),
   (28, 0.0014455208),
   (29, 0.0086777415),
   (30, 0.0014455208),
   (31, 0.0014455208),
   (32, 0.0014455208),
   (33, 0.0014455208),
   (34, 0.016579608),
   (35, 0.0014455208),
   (36, 0.0014455208),
   (37, 0.0014455208),
   (38, 0.0014455208),
   (39, 0.0014455208),
   (40, 0.0014455208),
   (41, 0.0014455208),
 